In [ ]:
import os
from langchain.chat_models import init_chat_model
from dataclasses import dataclass

os.environ["OPENAI_API_KEY"] = ""


In [53]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

def add_tool():
    """Tool that adds the numbers"""
    print("add_tool")

def multiply_tool():
    """Tool that multiplies the numbers"""
    
    print("multiply_tool")

def divide_tool():
    """Tool that performs the division"""
    print("divide_tool")
    



tool_node = ToolNode([add_tool, multiply_tool, divide_tool])

model = init_chat_model(model="openai:gpt-3.5-turbo")
model_with_tools = model.bind_tools([add_tool, multiply_tool, divide_tool])

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState)

# Define the two nodes we will cycle between
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_node)

builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")

graph = builder.compile()



In [54]:
graph.invoke({"messages": [{"role": "user", "content": "whats is the sum of 2 and 3?"}]})

add_tool


{'messages': [HumanMessage(content='whats is the sum of 2 and 3?', additional_kwargs={}, response_metadata={}, id='0255a6be-6d4b-4b16-9854-3a843bed6c85'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KXDufItE78p2BBPZ7muW9NTi', 'function': {'arguments': '{}', 'name': 'add_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CBldOxALL3iTLZ7zl1iwODH9NM1o4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--89caaea2-4261-41e7-9567-2cd2ecf1ea02-0', tool_calls=[{'name': 'add_tool', 'args': {}, 'id': 'call_KXDufItE78p2BBPZ7muW9NTi', 'type': 'tool_call'}], u

In [57]:
graph.invoke({"messages": [{"role": "user", "content": "whats is  2 x 4?"}]})

multiply_tool
multiply_tool
multiply_tool


{'messages': [HumanMessage(content='whats is  2 x 4?', additional_kwargs={}, response_metadata={}, id='0bdbd704-9442-435b-95e4-725271e961b5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_q3hWsYMMnmezOHKSLrEEgz2W', 'function': {'arguments': '{}', 'name': 'multiply_tool'}, 'type': 'function'}, {'id': 'call_1lJshOISYNuGZHbDYPueHlZy', 'function': {'arguments': '{}', 'name': 'multiply_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 79, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CBlrQOiJxzKVpztnB6v1rBuuwNK9t', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5db45a53-6b8e-4834-a056-66f19c4a367d

In [58]:
graph.invoke({"messages": [{"role": "user", "content": "whats is  2 multiplied by 4?"}]})

multiply_tool
multiply_tool


{'messages': [HumanMessage(content='whats is  2 multiplied by 4?', additional_kwargs={}, response_metadata={}, id='559bf8ff-7bdc-4380-a525-78de0a1f0b93'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3VwiejrJr5LuVbmfBYpY8gGA', 'function': {'arguments': '{}', 'name': 'multiply_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 80, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CBlrnwC9bFgtthutI2d7VJ507ujWl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f42a7c8f-36e3-466b-900f-b35428c0fdf9-0', tool_calls=[{'name': 'multiply_tool', 'args': {}, 'id': 'call_3VwiejrJr5LuVbmfBYpY8gGA', 'type': 'tool

In [59]:
graph.invoke({"messages": [{"role": "user", "content": "whats is product of 2 and 4?"}]})

multiply_tool
add_tool
multiply_tool


{'messages': [HumanMessage(content='whats is product of 2 and 4?', additional_kwargs={}, response_metadata={}, id='eb731c2b-f4d2-40b6-b28d-b924736e20c1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6zWBR5WnulLj7NQo2G4w8qVX', 'function': {'arguments': '{}', 'name': 'multiply_tool'}, 'type': 'function'}, {'id': 'call_cWsSrzv2bYcwpKLpTFoS85f3', 'function': {'arguments': '{}', 'name': 'add_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 80, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CBlsBaQs0ZOs04QXtCW84GVJHis6d', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d85a9ab0-0a0a-47bf-b3db-67bed

In [ ]:
import os
from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = ""

llm = init_chat_model("openai:gpt-3.5-turbo")

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tools = [tool]
tool.invoke("What's a 'node' in LangGraph?")

In [ ]:
from langchain_core.tools import tool

@tool
def get_name(location: str) -> str:
    """Execute when name is asked"""
    print("My name is langgraph")
    
@tool
def get_address(location: str) -> str:
    """Execute when address is asked"""
    print("My address is openai")    


tools = [tool]

In [61]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

NameError: name 'tools' is not defined